<a href="https://colab.research.google.com/github/neennera/GIBBIN-Web-Scraping/blob/main/WebScaping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---



# [Bug] Open Chrome using Selenium

In [ ]:
#Set up Chrome Driver
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
#ระบุตำแหน่งของเว็บปลายทาง
destination = "https://esc.kmutt.ac.th/index.php"

In [ ]:
#Open Chrome
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',options=chrome_options)

In [ ]:
#Open Website
driver = webdriver.Chrome('chromedriver',options=chrome_options)
driver.get(destination)

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


username_input = driver.find_element("id","username")
username_input.send_keys(gibbon_username)

pass_input = driver.find_element("id","password")
pass_input.send_keys(gibbon_password)

search_box = driver.find_element("id","loginForm")
search_box.submit()



# user_input = driver.find_element("username","q")

# [FIXED] open google chrom by selenium
support ubuntu 20.04 due to colab update ubanto version

In [1]:
destination = "https://esc.kmutt.ac.th/index.php"

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

options.headless = True

driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)

driver.get(destination)
print(driver.title)

<ipython-input-9-e12a53def73d>:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


ESC - ESC




---

# Web Scraping
- Get HTML source code using BeautifulSoup
- Read and transfer code to lecture data using HTMLParser

In [10]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


username_input = driver.find_element("id","username")
username_input.send_keys(gibbon_username)

pass_input = driver.find_element("id","password")
pass_input.send_keys(gibbon_password)

search_box = driver.find_element("id","loginForm")
search_box.submit()



# user_input = driver.find_element("username","q")

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [12]:
# driver.get(destination)
soup =  BeautifulSoup(driver.page_source, 'html.parser')

In [13]:
lecture = []

In [14]:
for i in range(0,5):
  get_div = driver.find_element('xpath','//*[@id="ttWrapper"]/table/tbody/tr[3]/td[' + str(i+2) + ']')
  lecture.append(get_div.get_attribute('innerHTML'))

In [15]:
from html.parser import HTMLParser

class MyHTMLParser(HTMLParser):
  def __init__(self):
    HTMLParser.__init__(self)
    HTMLParser.unwanted_tag = ['div','i','a','br']
    HTMLParser.lectuteInWeek = []
    HTMLParser.day = 0

  def GetLectuteInWeek(self, HTMLtext):
    for i in range(0,5):
      HTMLParser.lectuteInWeek.append([])
      parser.feed(HTMLtext[i])
      HTMLParser.day += 1
    return HTMLParser.lectuteInWeek
  
  def handle_data(self, data):
    # print("Encountered some data  :", data)
    HTMLParser.lectuteInWeek[HTMLParser.day].append(data)

  # def handle_starttag(self, tag, attrs):
  #   if not(tag in HTMLParser.unwanted_tag) :
  #     print("Encountered a start tag:", tag)

  # def handle_endtag(self, tag):
  #   if not(tag in HTMLParser.unwanted_tag) :
  #     print("Encountered an end tag :", tag)

parser = MyHTMLParser()
lectureInWeek = parser.GetLectuteInWeek(lecture)

# Analyse Data

In [16]:
temp = lectureInWeek.copy()

In [17]:
class lecture:
  def __init__(self, day=-1 , start=-1, end=-1, subject="none", className="none"):
    self.day = day
    self.start = start
    self.end = end
    self.className = className
    self.subject = subject

In [18]:
lectureData = []

In [19]:
for day in range(0,5):
  temp = lectureInWeek[day].copy()
  i=0
  while i< len(temp): 
    if temp[i][0:4] == "ESC ":  
      
      #get subject and class name
      subject = temp[i][0:3] + temp[i][4:-2]
      if subject[-1] == ".":
        subject = subject[:-1]
      className = temp[i+1]

      # get time
      timeIndex = temp.index(subject) +1
      time = temp[timeIndex]
      start = time[0:5]
      end = time[-5:]
      
      # remove read class
      temp.remove(subject)
      temp.remove(time)

      #append data
      print(str(day) + "  " + subject + " ; " + className + " ; ", start + " " + end)
      lectureData.append(lecture(day,start,end,subject,className))
      continue

    if temp[i] == "School Closed":
      print(str(day) + "  " + "School Closed")
      lectureData.append(lecture(day, "08:00" , "16:00", "School Closed"))
      break
    
    i+=1

0  ESC625 ; Pollution I ;  09:00 11:00
0  ESC624 ; Nervous System II ;  12:00 14:00
0  ESC652 ; Unit 10 Opener: Social Living  ;  14:00 15:00
0  ESC642 ; ทดสอบทักษะการทำคะแนน(ตบ) ;  15:00 16:00
1  ESC625 ; Pollution II ;  09:00 11:00
1  ESC625 ; 6 ตุลา 19 ;  12:00 14:00
1  ESC652 ; Quiz 2 & Unit 10A: A Co-Operative Society  ;  14:00 16:00
1  ESC621 ; (EXT) การเข้ารหัสถอดรหัส 2 ;  20:00 22:00
2  ESC624 ; Gifted Hand Movie ;  09:00 11:00
2  ESC621 ; การเข้ารหัสถอดรหัส 1 ;  12:00 14:00
2  ESC625 ; Pollution III ;  14:00 16:00
2  ESC625 ; (EXT) Self-study ;  16:00 18:00
3  ESC625 ; การเมืองไทยหลังยุค 6 ตุลา 2519-พฤษภาทมิฬ 2535 ;  09:00 11:00


ValueError: ignored